# 文章目录
[IO编程](#1)
- 文件读写
- 操作文件和目录
- 序列化
- 使用Json

[进程和线程](#2)
- 暂时没学完

[正则表达式](#3)

# IO编程
<a id='1'></a>

### 读文件


In [8]:
f = open('data.txt','r')

如果文件不存在，open()函数就会抛出一个IOError的错误，并且给出错误码和详细的信息告诉你文件不存在：

In [3]:
f.read()

'1 2 3 4\n2 3 4 5\n'

最后一步是调用close()方法关闭文件。文件使用完毕后必须关闭，因为文件对象会占用操作系统的资源，并且操作系统同一时间能打开的文件数量也是有限的

In [4]:
f.close()

In [9]:
#Python引入了with语句来自动帮我们调用close()方法：
with open('data.txt','r') as f:
    print f.read()


hello,world!


调用read()会一次性读取文件的全部内容，如果文件有10G，内存就爆了，所以，要保险起见，可以反复调用read(size)方法，每次最多读取size个字节的内容。另外，调用readline()可以每次读取一行内容，调用readlines()一次读取所有内容并按行返回list。因此，要根据需要决定怎么调用。

如果文件很小，read()一次性读取最方便；如果不能确定文件大小，反复调用read(size)比较保险；如果是配置文件，调用readlines()最方便

### 写文件 

写文件和读文件是一样的，唯一区别是调用open()函数时，传入标识符'w'或者'wb'表示写文本文件或写二进制文件

In [6]:
f = open('data.txt','w')
f.write('\nhello,world!')

In [7]:
f.close()

你可以反复调用write()来写入文件，但是务必要调用f.close()来关闭文件。当我们写文件时，操作系统往往不会立刻把数据写入磁盘，而是放到内存缓存起来，空闲的时候再慢慢写入。只有调用close()方法时，操作系统才保证把没有写入的数据全部写入磁盘。忘记调用close()的后果是数据可能只写了一部分到磁盘，剩下的丢失了。所以，还是用with语句来得保险

### 操作文件和目录 

果我们要操作文件、目录，可以在命令行下面输入操作系统提供的各种命令来完成。比如dir、cp等命令。

如果要在Python程序中执行这些目录和文件的操作怎么办？其实操作系统提供的命令只是简单地调用了操作系统提供的接口函数，Python内置的os模块也可以直接调用操作系统提供的接口函数

In [10]:
import os
os.name #操作系统名字

'nt'

如果是posix，说明系统是Linux、Unix或Mac OS X，如果是nt，就是Windows系统

In [11]:
#环境变量
os.environ

{'TMP': 'C:\\Users\\HAIER-~1\\AppData\\Local\\Temp', 'COMPUTERNAME': 'LAPTOP-809QM52A', 'USERDOMAIN': 'LAPTOP-809QM52A', 'PROCESSOR_ARCHITECTURE': 'AMD64', 'PSMODULEPATH': 'C:\\Program Files\\WindowsPowerShell\\Modules;C:\\Windows\\system32\\WindowsPowerShell\\v1.0\\Modules', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'PROCESSOR_IDENTIFIER': 'Intel64 Family 6 Model 142 Stepping 9, GenuineIntel', 'PROGRAMFILES': 'C:\\Program Files', 'PROCESSOR_REVISION': '8e09', 'SYSTEMROOT': 'C:\\Windows', 'PATH': 'C:\\Users\\haier-003\\Anaconda2\\Library\\bin;C:\\Users\\haier-003\\Anaconda2\\Library\\bin;C:\\Users\\haier-003\\Anaconda2;C:\\Users\\haier-003\\Anaconda2\\Library\\mingw-w64\\bin;C:\\Users\\haier-003\\Anaconda2\\Library\\usr\\bin;C:\\Users\\haier-003\\Anaconda2\\Library\\bin;C:\\Users\\haier-003\\Anaconda2\\Scripts;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\;C:\\Program Files (x86)\\Windows Kits\\10\\Windows P

In [12]:
#查看当前目录的绝对路径：
os.path.abspath('.')

'C:\\Users\\haier-003'

In [14]:
# 在某个目录下创建一个新目录，
# 首先把新目录的完整路径表示出来:
os.path.join('/Users/Users', 'testdir')

'/Users/Users\\testdir'

In [15]:
#列出当前目录下的所有目录：
[x for x in os.listdir('.') if os.path.isdir(x)]

['.android',
 '.AndroidStudio2.2',
 '.AndroidStudio2.3',
 '.conda',
 '.gradle',
 '.idlerc',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.matplotlib',
 '.spyder',
 '.vscode',
 'Anaconda2',
 'AppData',
 'Application Data',
 'Contacts',
 'Cookies',
 'Desktop',
 'Documents',
 'Downloads',
 'Evernote',
 'Favorites',
 'IntelGraphicsProfiles',
 'Links',
 'Local Settings',
 'Music',
 'My Documents',
 'NetHood',
 'OneDrive',
 'Pictures',
 'PrintHood',
 'Recent',
 'Saved Games',
 'Searches',
 'SendTo',
 'Templates',
 'Videos',
 '\xa1\xb8\xbf\xaa\xca\xbc\xa1\xb9\xb2\xcb\xb5\xa5']

### 序列化 

可以随时修改变量，比如把name改成'Bill'，但是一旦程序结束，变量所占用的内存就被操作系统全部回收。如果没有把修改后的'Bill'存储到磁盘上，下次重新运行程序，变量又被初始化为'Bob'。

我们把变量从内存中变成可存储或传输的过程称之为序列化，在Python中叫pickling，在其他语言中也被称之为serialization，marshalling，flattening等等，都是一个意思。

序列化之后，就可以把序列化后的内容写入磁盘，或者通过网络传输到别的机器上。

反过来，把变量内容从序列化的对象重新读到内存里称之为反序列化，即unpickling。

Python提供两个模块来实现序列化：cPickle和pickle。这两个模块功能是一样的，区别在于cPickle是C语言写的，速度快，pickle是纯Python写的，速度慢，跟cStringIO和StringIO一个道理。用的时候，先尝试导入cPickle，如果失败，再导入pickle

In [16]:
try:
    import cPickle as pickle
except ImportError:
    import pickle

In [17]:
d = dict(name='Bob', age=20,socre=80)

In [18]:
pickle.dumps(d)

"(dp1\nS'age'\np2\nI20\nsS'socre'\np3\nI80\nsS'name'\np4\nS'Bob'\np5\ns."

pickle.dumps()方法把任意对象序列化成一个str，然后，就可以把这个str写入文件。或者用另一个方法pickle.dump()直接把对象序列化后写入一个file-like Object

In [20]:
f = open('dump.txt','wb')

In [21]:
pickle.dump(d,f)
f.close()

看看写入的dump.txt文件，一堆乱七八糟的内容，这些都是Python保存的对象内部信息。

当我们要把对象从磁盘读到内存时，可以先把内容读到一个str，然后用pickle.loads()方法反序列化出对象，也可以直接用pickle.load()方法从一个file-like Object中直接反序列化出对象。我们打开另一个Python命令行来反序列化刚才保存的对象

In [22]:
f = open('dump.txt','rb')
d = pickle.load(f)
f.close()

In [23]:
d

{'age': 20, 'name': 'Bob', 'socre': 80}

### Json 

如果我们要在不同的编程语言之间传递对象，就必须把对象序列化为标准格式，比如XML，但更好的方法是序列化为JSON，因为JSON表示出来就是一个字符串，可以被所有语言读取，也可以方便地存储到磁盘或者通过网络传输。JSON不仅是标准格式，并且比XML更快，而且可以直接在Web页面中读取，非常方便。

In [24]:
import json
d = dict(name='Bob',age=20,score=88)
json.dumps(d)

'{"age": 20, "score": 88, "name": "Bob"}'

In [25]:
d

{'age': 20, 'name': 'Bob', 'score': 88}

In [26]:
json_str = '{"age": 20, "score": 88, "name": "Bob"}'
json.loads(json_str)

{u'age': 20, u'name': u'Bob', u'score': 88}

Python的dict对象可以直接序列化为JSON的{}，不过，很多时候，我们更喜欢用class表示对象，比如定义Student类，然后序列化

In [27]:
class Student(object):
    def __init__(self,name,age,score):
        self.name = name
        self.age = age
        self.score = score
        
s = Student('Bob',20,99)
print(json.dumps(s))

TypeError: <__main__.Student object at 0x00000000061F6F60> is not JSON serializable

错误的原因是Student对象不是一个可序列化为JSON的对象。

前面的代码之所以无法把Student类实例序列化为JSON，是因为默认情况下，dumps()方法不知道如何将Student实例变为一个JSON的{}对象。

可选参数default就是把任意一个对象变成一个可序列为JSON的对象，我们只需要为Student专门写一个转换函数，再把函数传进去即可

In [28]:
def student2dict(std):
    return{
        'name':std.name,
        'age':std.age,
        'score':std.score
    }
print(json.dumps(s,default=student2dict))

{"age": 20, "score": 99, "name": "Bob"}


不过，下次如果遇到一个Teacher类的实例，照样无法序列化为JSON。我们可以偷个懒，把任意class的实例变为dict

In [29]:
print(json.dumps(s, default=lambda obj: obj.__dict__))

{"age": 20, "score": 99, "name": "Bob"}


因为通常class的实例都有一个\_\_dict\_\_属性，它就是一个dict，用来存储实例变量。也有少数例外，比如定义了\_\_slots\_\_的class。

同样的道理，如果我们要把JSON反序列化为一个Student对象实例，loads()方法首先转换出一个dict对象，然后，我们传入的object_hook函数负责把dict转换为Student实例

In [30]:
def dict2student(d):
    return Student(d['name'], d['age'], d['score'])

json_str = '{"age": 20, "score": 88, "name": "Bob"}'
print(json.loads(json_str, object_hook=dict2student))

<a id='2'></a>
# 进程和线程

Unix/Linux操作系统提供了一个fork()系统调用，它非常特殊。普通的函数调用，调用一次，返回一次，但是fork()调用一次，返回两次，因为操作系统自动把当前进程（称为父进程）复制了一份（称为子进程），然后，分别在父进程和子进程内返回。

子进程永远返回0，而父进程返回子进程的ID。这样做的理由是，一个父进程可以fork出很多子进程，所以，父进程要记下每个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID。

Python的os模块封装了常见的系统调用，其中就包括fork，可以在Python程序中轻松创建子进程

由于Windows没有fork调用，难道在Windows上无法用Python编写多进程的程序？

由于Python是跨平台的，自然也应该提供一个跨平台的多进程支持。multiprocessing模块就是跨平台版本的多进程模块。

multiprocessing模块提供了一个Process类来代表一个进程对象，下面的例子演示了启动一个子进程并等待其结束：


In [34]:
from multiprocessing import Process
import os

# 子进程要执行的代码
def run_proc(name):
    print 'Run child process %s (%s)...' % (name, os.getpid())

if __name__=='__main__':
    print 'Parent process %s.' % os.getpid()
    p = Process(target=run_proc, args=('test',))
    print 'Process will start.'
    p.start()
    p.join()
    print 'Process end.'

Parent process 5520.
Process will start.
Process end.


### 线程池POOL 

In [ ]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print 'Run task %s (%s)...' % (name, os.getpid())
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print 'Task %s runs %0.2f seconds.' % (name, (end - start))

if __name__=='__main__':
    print 'Parent process %s.' % os.getpid()
    p = Pool()
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print 'Waiting for all subprocesses done...'
    p.close()
    p.join()
    print 'All subprocesses done.'

Parent process 11628.
Waiting for all subprocesses done...


In [3]:
import time, threading

# 新线程执行的代码:
def loop():
    print 'thread %s is running...' % threading.current_thread().name
    n = 0
    while n < 5:
        n = n + 1
        print 'thread %s >>> %s' % (threading.current_thread().name, n)
        time.sleep(1)
    print 'thread %s ended.' % threading.current_thread().name

print 'thread %s is running...' % threading.current_thread().name
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print 'thread %s ended.' % threading.current_thread().name

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.


# 正则表达式
<a id='3'></a>

正则表达式是一种用来匹配字符串的武器。它的设计思想是用一种描述性语言来给字符串定义一个规则，凡是符合规则的字符串我们就认为它匹配了，否则该字符串是不合法的

所以我们判断一个字符串是否合法的Email的方法是：
- 创建一个匹配Email的正则表达式
- 用该正则表达式去匹配用户的输入来判断是否合法

因为正则表达式也是用字符串表示的，所以，我们首先要了解如何用字符来描述字符。

在正则表达式种，如果直接给出字符就是精确匹配。用 \d 可以匹配一个数字，\w可以匹配一个字母或者数字，所以：
- ’00\d’可以匹配‘007’但是无法匹配‘00A’
- ‘\d\d\d’可以匹配‘010’
- ‘\w\w\d’可以匹配‘py3’
‘.’可以匹配任意字符所以：
- ‘py.’可以匹配‘pyc’,‘py0’‘pyl’等等

要匹配变长的字符，在正则表达式种，用*表示任意个字符，用+表示至少一个字符，？表示0或者1个字符，用{n}表示n个字符，用{n,m}表示n-m个字符：

\d{3}\s+\d(3,8)表示：
- \d{3}表示匹配个数字，例如‘010’
- \s可以匹配一个空格，也包括Tab等空白字符，所以\s+表示至少有一个空格
- \d{3，8}表示3-8个数字，例如{1234567}

所以上面的正则表达式可以匹配人一个空格隔开的带区号的电话号码

如果要匹配‘010-12345’这样的电话号码呢？由于‘-’是特殊字符，在正则表达式种，要用'\'转移，所以它的正则是\d{3}\-\d{3-8}

但是任然无法匹配‘010 - 12345’,因为带有空格，所以我们需要更复杂的匹配方式

### 进阶

要做更精确的匹配，可以用[]表示范围，比如：
- [0-9a-zA-Z\_]可以匹配一个数字、字母或者下划线
- [0-9a-zA-Z\_]+可以匹配至少由一个数字，字母或者下划线组成的字符串比如：‘a100’,‘0_z’,‘Py3000’等等
- [a-zA-Z\_][0-9a-zA-Z\_]* 可以匹配由字母或者下划线开头，后接任意一个由数字，字母或者下划线组成的字符串，也就是Pythom合法的变量
- [a-zA-Z\_][0-9A-Za-z\_][0,19]更精确的限制的变量长度是1-20个字符（前面1个字符+后面最多19个字符）
- A|B可以匹配A或B，所以(P|p)ython可以匹配‘Python’或者‘python’
- ^表示行的开头，^\d表示必须以数字开头
- $表示行结束，(\d$$)表示必须以字符串结束

你可能注意到了，py也可以匹配'python'，但是加上^py$就变成了整行匹配，就只能匹配'py'了。

### Re模块
有了准备知识，我们就可以在Python中使用正则表达式了。Python提供re模块，包含所有正则表达式的功能。由于Python的字符串本身也用\转义，所以要特别注意

In [3]:
s = 'ABC\\-001' #Python的字符串
#对应的正则表达式位： 'ABC\-001'
print s

ABC\-001


In [4]:
#因此我们建议用Python的r前缀，就不用考虑转义的问题
s = r'ABC\-001'
print s

ABC\-001


In [5]:
#看看如何判断正则表达式是否匹配
import re
re.match(r'^\d{3}\-\d{3,8}','010-12345')

In [7]:
re.match(r'^\d{3}\-\d{3,8}','010 12345')

match()方法判断是否匹配，如果匹配成功，返回一个Match对象，否则返回None。常见的判断方法：

In [10]:
test = '用户输入的字符串'
if re.match(r'正则表达式','正则表达式'):
    print 'ok'
else:
    print 'failed'

ok


### 切分字符串

用正则表达式切分字符串比用固定的字符更加灵活

In [11]:
#正常的切分代码
'a b      c'.split(' ')

['a', 'b', '', '', '', '', '', 'c']

In [12]:
#正则表达式
re.split(r'\s+', 'a b      c')

['a', 'b', 'c']

### 分组

除了简单的判断是否匹配之外，正则表达式还有提取子串的强大功能。用()表示的就是要提取分组。比如：
 ^(\d{3})-(\d{3,8})分别定义了两个组，可以直接从匹配的字符串种提取出区号和本地号码

In [13]:
m = re.match(r'^(\d{3})-(\d{3,8})$','010-12345')
m

In [14]:
m.group(0)

'010-12345'

In [15]:
m.group(1)

'010'

In [16]:
m.group(2)

'12345'

如果正则表达式种定义了组，就可以在match对象上用group方法提取出子串来

来看一个凶残的例子：

In [18]:
t = '19:05:30'
m = re.match(r'^(0[0-9]|1[0-9]|2[0-3]|[0-9])\:(0[0-9]|1[0-9]|2[0-9]|3[0-9]|4[0-9]|5[0-9]|[0-9])\:(0[0-9]|1[0-9]|2[0-9]|3[0-9]|4[0-9]|5[0-9]|[0-9])$', t)


In [19]:
m.groups()

('19', '05', '30')

### 贪婪匹配

正则匹配默认的是贪婪匹配，也就是匹配尽可能多的字符：

In [20]:
re.match(r'^(\d+)(0*)$','1023400').groups()

('1023400', '')

由于\d+采用贪婪匹配，直接把后面的0全部匹配了，结果0*只能匹配空字符串了

必须让\d+采用非贪婪匹配，才能把后面的0匹配出来，加个？就能让\d+采用非贪婪匹配

In [ ]:
re,match(r'^(\d+?)(0*)&')